In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models, similarities, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
with open('../podcasts/podcasts/jhs.pickle', 'rb') as read_jhs:
    jhs_df = pickle.load(read_jhs)

In [3]:
with open('../podcasts/podcasts/aoc.pickle', 'rb') as read_aoc:
    pdf_df = pickle.load(read_aoc)

In [4]:
df = pd.concat([jhs_df,pdf_df],ignore_index=True)

In [8]:
df.head()

,url,title,transcript,source,cleaned
0,https://www.jordanharbinger.com/should-a-cheat...,469: Should a Cheater Get a Second Chance? | F...,"[<p><b>Jordan Harbinger: </b><span style=""font...",JHS,Jordan Harbinger: Welcome to Feedback Friday. ...
1,https://www.jordanharbinger.com/going-to-north...,435: Going to North Korea: Part One | Stereo S...,[ Welcome to the show. I'm Jordan Harbinger. O...,JHS,Welcome to the show. I'm Jordan Harbinger. On...
2,https://www.jordanharbinger.com/stuart-ritchie...,436: Stuart Ritchie | The Science Fictions Und...,[ This podcast is brought to you by Microsoft ...,JHS,This podcast is brought to you by Microsoft T...
3,https://www.jordanharbinger.com/koch-and-hooks...,437: Charles Koch & Brian Hooks | Bottom-Up So...,"[ Coming up on The Jordan Harbinger Show. , [0...",JHS,Coming up on The Jordan Harbinger Show. It's...
4,https://www.jordanharbinger.com/how-to-straigh...,438: How to Straighten Out Your Crooked Boss |...,"[<p><b>Jordan Harbinger: </b><span style=""font...",JHS,Jordan Harbinger: Welcome to Feedback Friday. ...


In [17]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
df['transcript'] = df.transcript.map(alphanumeric).map(punc_lower)
df.head()

TypeError: expected string or bytes-like object

In [117]:
#(541, 44569) With only stop_words='english'
cv1 = CountVectorizer(stop_words='english',strip_accents='unicode',min_df=200,max_features=600)
countvect = cv1.fit_transform(df['cleaned'])
new_df = pd.DataFrame(countvect.toarray(), columns=cv1.get_feature_names(),index=df['title'])

In [ ]:
with open('countvect.pickle', 'wb') as cv_pickle:
    pickle.dump(countvect, cv_pickle)

In [ ]:
#no max/min dfs was 44569, .5/5 was 16640, .4/10=10568
new_df.shape

In [107]:
tfidf1 = TfidfVectorizer(stop_words='english',strip_accents='unicode',max_df=.20,min_df=100,max_features=30)
_tfidf1 = tfidf1.fit_transform(df['cleaned'])
new_df2 = pd.DataFrame(_tfidf1.toarray(), columns=tfidf1.get_feature_names(),index=df['title'])

In [13]:
with open('tfidf.pickle', 'wb') as tfidf_pickle:
    pickle.dump(_tfidf1, tfidf_pickle)

In [24]:
new_df2.shape

(541, 44569)

In [17]:
df[df['cleaned'].str.contains('zzjjhjz')]

,url,title,transcript,source,cleaned
266,https://www.jordanharbinger.com/nir-eyal-contr...,250: Nir Eyal | Control Your Attention and Cho...,"[<p><b>Jordan Harbinger:</b><span style=""font-...",JHS,Welcome to the show. I'm Jordan Harbinger. As...


In [18]:
print(df.loc[266,'cleaned'])

 Welcome to the show. I'm Jordan Harbinger. As always, I'm here with producer, Jason DeFillippo. On The Jordan Harbinger Show, we decode the stories, secrets, and skills of the world's most brilliant and interesting people and turn their wisdom into practical advice that you can use to impact your own life and those around you. Chase Jarvis, he's an award-winning artist, entrepreneur ,and one of the most influential photographers of the past decade and he's a friend of mine, which always works well here on the show. He's worked for huge brands like Apple and Red Bull. However, he's also been in some big slumps, lost his creativity, found it again, sacrificed it on the altar of Silicon Valley capitalism like many of us have, and then reclaimed it. Not a bad ride. Today, we'll discuss some of the most common barriers to creativity and our pursuit of a creative outlet or career, whether that's work or family obligations or just a lack of know-how. Of course, we'll also show you how to wor

In [108]:
lsa1 = TruncatedSVD(5)
cv_topic = lsa1.fit_transform(countvect)
lsa1.explained_variance_ratio_

array([0.19635276, 0.10732225, 0.07132552, 0.05106534, 0.04578245])

In [109]:
cv_topic_word = pd.DataFrame(lsa1.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = cv1.get_feature_names())
cv_topic_word

,bottle,boyfriend,camp,cancer,crack,crisis,domain,fbi,friendship,gas,...,population,professor,race,rights,salary,suffering,tank,technique,threat,toxic
component_1,0.022,0.013,0.025,0.050,0.015,0.023,0.010,0.049,0.010,0.030,...,0.016,0.014,0.013,0.014,0.015,0.994,0.012,0.022,0.015,0.016
component_2,0.065,0.064,0.064,0.142,0.097,0.086,0.071,0.873,0.046,0.060,...,0.064,0.057,0.048,0.118,0.059,-0.087,0.065,0.111,0.108,0.056
component_3,0.065,0.086,0.078,0.824,0.046,0.238,0.060,-0.328,0.048,0.064,...,0.089,0.095,0.107,0.026,0.102,-0.061,0.064,0.033,0.067,0.123
component_4,0.056,0.117,0.245,-0.514,0.082,0.543,0.095,-0.221,0.079,0.092,...,0.128,0.059,0.137,0.067,0.089,-0.022,0.084,0.055,0.219,0.067
component_5,-0.077,-0.166,-0.292,0.086,-0.069,0.767,-0.155,0.132,-0.064,-0.038,...,-0.013,-0.032,-0.135,-0.049,-0.136,0.014,-0.067,-0.055,-0.069,-0.129


In [110]:
lsa2 = TruncatedSVD(5)
tfidf_topic = lsa2.fit_transform(_tfidf1)
lsa2.explained_variance_ratio_

array([0.00828699, 0.05832786, 0.05350391, 0.04850087, 0.04559039])

In [111]:
tfidf_topic_word = pd.DataFrame(lsa2.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = tfidf1.get_feature_names())
tfidf_topic_word

,bottle,boyfriend,camp,cancer,crack,crisis,domain,fbi,friendship,gas,...,population,professor,race,rights,salary,suffering,tank,technique,threat,toxic
component_1,0.169,0.212,0.199,0.206,0.172,0.179,0.166,0.250,0.213,0.170,...,0.174,0.184,0.152,0.162,0.209,0.162,0.208,0.187,0.159,0.195
component_2,-0.044,-0.352,0.004,-0.176,0.075,0.019,0.009,0.636,-0.323,0.036,...,0.066,-0.061,-0.006,0.065,-0.248,-0.095,-0.091,0.055,0.145,-0.258
component_3,-0.019,0.451,-0.024,-0.472,-0.191,-0.148,0.010,0.409,0.282,-0.046,...,-0.180,-0.163,-0.202,0.035,0.152,-0.176,0.084,0.085,-0.008,0.080
component_4,0.069,0.067,0.210,0.121,0.000,0.140,-0.202,0.065,0.589,0.065,...,0.069,-0.029,-0.014,-0.035,-0.519,0.101,-0.248,-0.186,0.113,0.029
component_5,-0.026,0.041,0.167,-0.275,0.139,0.016,-0.089,-0.183,-0.082,0.138,...,-0.017,-0.098,0.040,-0.043,-0.448,-0.009,0.744,0.097,-0.039,0.042


In [112]:
nmf1_model = NMF(5,max_iter=20000)
nmf1_topic = nmf1_model.fit_transform(countvect)

/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


In [113]:
nmf1_topic_word = pd.DataFrame(nmf1_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = cv1.get_feature_names())
nmf1_topic_word

,bottle,boyfriend,camp,cancer,crack,crisis,domain,fbi,friendship,gas,...,population,professor,race,rights,salary,suffering,tank,technique,threat,toxic
component_1,0.104,0.000,0.052,0.000,0.017,0.000,0.000,0.000,0.012,0.188,...,0.029,0.020,0.000,0.012,0.009,10.315,0.004,0.089,0.000,0.016
component_2,0.243,0.000,0.000,0.000,0.521,0.000,0.130,9.374,0.081,0.162,...,0.092,0.113,0.000,0.826,0.000,0.000,0.186,0.759,0.276,0.000
component_3,0.280,0.045,0.000,8.316,0.071,0.000,0.000,0.000,0.023,0.141,...,0.186,0.475,0.182,0.055,0.284,0.000,0.138,0.113,0.000,0.539
component_4,1.008,1.791,2.547,0.000,1.187,0.000,1.605,0.000,0.972,1.017,...,1.155,0.885,1.712,0.980,1.564,0.000,1.119,0.975,1.994,1.428
component_5,0.000,0.000,0.000,0.000,0.042,6.947,0.000,0.000,0.089,0.275,...,0.559,0.259,0.000,0.084,0.000,0.000,0.119,0.046,0.336,0.000


In [114]:
nmf2_model = NMF(5,max_iter=2000)
nmf2_topic = nmf2_model.fit_transform(_tfidf1)

/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


In [115]:
nmf2_topic_word = pd.DataFrame(nmf2_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = tfidf1.get_feature_names())
nmf2_topic_word

,bottle,boyfriend,camp,cancer,crack,crisis,domain,fbi,friendship,gas,...,population,professor,race,rights,salary,suffering,tank,technique,threat,toxic
component_1,0.251,0.099,0.000,0.000,0.000,0.000,0.690,0.00,0.000,0.000,...,0.000,0.109,0.000,0.315,2.211,0.126,0.000,0.369,0.000,0.017
component_2,0.053,0.000,0.091,0.000,0.038,0.034,0.140,1.74,0.000,0.086,...,0.044,0.000,0.000,0.245,0.000,0.000,0.000,0.284,0.331,0.000
component_3,0.301,0.000,0.437,0.808,0.571,0.552,0.150,0.00,0.000,0.388,...,0.588,0.591,0.525,0.224,0.000,0.487,0.000,0.068,0.308,0.155
component_4,0.196,1.172,0.241,0.000,0.000,0.072,0.005,0.00,1.333,0.092,...,0.000,0.009,0.000,0.034,0.000,0.081,0.000,0.031,0.063,0.765
component_5,0.000,0.064,0.064,0.000,0.018,0.008,0.034,0.00,0.000,0.176,...,0.000,0.004,0.025,0.019,0.000,0.000,1.919,0.444,0.000,0.022


In [118]:
corpus1 = matutils.Sparse2Corpus(countvect)

In [119]:
id2word1 = dict((v, k) for k, v in cv1.vocabulary_.items())

In [22]:
len(id2word1)

16640

In [120]:
lda1 = models.LdaModel(corpus=corpus1, num_topics=3, id2word=id2word1, passes=50,update_every=1)

2021-02-20 22:53:27,028 : INFO : using symmetric alpha at 0.3333333333333333
2021-02-20 22:53:27,033 : INFO : using symmetric eta at 0.3333333333333333
2021-02-20 22:53:27,035 : INFO : using serial LDA version on this node
2021-02-20 22:53:27,050 : INFO : running online (multi-pass) LDA training, 3 topics, 50 passes over the supplied corpus of 600 documents, updating model once every 600 documents, evaluating perplexity every 600 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-20 22:54:07,214 : INFO : -6.819 per-word bound, 112.9 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 22:54:07,215 : INFO : PROGRESS: pass 0, at document #600/600
2021-02-20 22:54:29,852 : INFO : topic #0 (0.333): 0.004*"dad" + 0.004*"hit" + 0.003*"eat" + 0.003*"probably" + 0.003*"credit" + 0.003*"getting" + 0.003*"know" + 0.003*"doing" + 0.003*"came" + 0.003*"networking"
2021-02-20 22:54:29,855 : INFO : topic #1 (0.333): 0.003*"hold" + 0.003

2021-02-20 23:03:24,724 : INFO : topic #1 (0.333): 0.004*"terms" + 0.004*"age" + 0.004*"seeing" + 0.004*"type" + 0.004*"self" + 0.003*"supporting" + 0.003*"hold" + 0.003*"second" + 0.003*"seven" + 0.003*"turn"
2021-02-20 23:03:24,726 : INFO : topic #2 (0.333): 0.005*"personal" + 0.004*"recommend" + 0.004*"percent" + 0.004*"dad" + 0.004*"eat" + 0.004*"pain" + 0.004*"sitting" + 0.004*"finding" + 0.004*"episode" + 0.003*"places"
2021-02-20 23:03:24,729 : INFO : topic diff=0.016185, rho=0.316228
2021-02-20 23:04:15,735 : INFO : -6.261 per-word bound, 76.7 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 23:04:15,736 : INFO : PROGRESS: pass 9, at document #600/600
2021-02-20 23:04:47,203 : INFO : topic #0 (0.333): 0.005*"dad" + 0.004*"hit" + 0.004*"came" + 0.004*"getting" + 0.004*"probably" + 0.003*"know" + 0.003*"guests" + 0.003*"kinds" + 0.003*"hour" + 0.003*"fight"
2021-02-20 23:04:47,205 : INFO : topic #1 (0.333): 0.004*"terms" + 0.004*"age" 

2021-02-20 23:14:18,244 : INFO : topic #1 (0.333): 0.005*"terms" + 0.005*"self" + 0.005*"age" + 0.005*"second" + 0.005*"supporting" + 0.004*"seeing" + 0.004*"turn" + 0.004*"type" + 0.004*"talk" + 0.004*"amazing"
2021-02-20 23:14:18,245 : INFO : topic #2 (0.333): 0.005*"personal" + 0.004*"eat" + 0.004*"recommend" + 0.004*"percent" + 0.004*"pain" + 0.004*"dad" + 0.004*"remember" + 0.004*"career" + 0.004*"episode" + 0.004*"sitting"
2021-02-20 23:14:18,246 : INFO : topic diff=0.009166, rho=0.229416
2021-02-20 23:15:24,080 : INFO : -6.254 per-word bound, 76.3 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 23:15:24,083 : INFO : PROGRESS: pass 18, at document #600/600
2021-02-20 23:15:44,452 : INFO : topic #0 (0.333): 0.005*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"getting" + 0.004*"group" + 0.004*"hour" + 0.004*"fight" + 0.003*"came" + 0.003*"middle" + 0.003*"speaking"
2021-02-20 23:15:44,453 : INFO : topic #1 (0.333): 0.005*"terms" + 0.00

2021-02-20 23:25:10,787 : INFO : topic #1 (0.333): 0.006*"terms" + 0.005*"supporting" + 0.005*"second" + 0.005*"self" + 0.005*"age" + 0.005*"talk" + 0.005*"turn" + 0.005*"seeing" + 0.005*"thanks" + 0.005*"type"
2021-02-20 23:25:10,790 : INFO : topic #2 (0.333): 0.005*"personal" + 0.005*"eat" + 0.005*"pain" + 0.005*"recommend" + 0.005*"percent" + 0.004*"remember" + 0.004*"career" + 0.004*"dad" + 0.004*"episode" + 0.004*"actually"
2021-02-20 23:25:10,791 : INFO : topic diff=0.006235, rho=0.188982
2021-02-20 23:25:57,822 : INFO : -6.250 per-word bound, 76.1 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 23:25:57,824 : INFO : PROGRESS: pass 27, at document #600/600
2021-02-20 23:26:16,731 : INFO : topic #0 (0.333): 0.005*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"group" + 0.003*"middle" + 0.003*"hold" + 0.003*"getting" + 0.003*"body" + 0.003*"speaking" + 0.003*"hour"
2021-02-20 23:26:16,732 : INFO : topic #1 (0.333): 0.006*"terms" + 0.005

2021-02-20 23:34:58,295 : INFO : topic #1 (0.333): 0.006*"terms" + 0.006*"supporting" + 0.006*"second" + 0.006*"age" + 0.005*"self" + 0.005*"talk" + 0.005*"thanks" + 0.005*"robert" + 0.005*"turn" + 0.005*"amazing"
2021-02-20 23:34:58,299 : INFO : topic #2 (0.333): 0.005*"personal" + 0.005*"eat" + 0.005*"pain" + 0.005*"recommend" + 0.005*"percent" + 0.004*"remember" + 0.004*"career" + 0.004*"actually" + 0.004*"episode" + 0.004*"dad"
2021-02-20 23:34:58,301 : INFO : topic diff=0.004481, rho=0.164399
2021-02-20 23:35:43,790 : INFO : -6.248 per-word bound, 76.0 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 23:35:43,791 : INFO : PROGRESS: pass 36, at document #600/600
2021-02-20 23:35:49,698 : INFO : topic #0 (0.333): 0.005*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"hold" + 0.004*"body" + 0.004*"middle" + 0.004*"group" + 0.003*"speaking" + 0.003*"issue" + 0.003*"sort"
2021-02-20 23:35:49,700 : INFO : topic #1 (0.333): 0.006*"terms" + 0.00

2021-02-20 23:43:04,225 : INFO : topic #1 (0.333): 0.006*"terms" + 0.006*"supporting" + 0.006*"age" + 0.006*"second" + 0.006*"talk" + 0.006*"self" + 0.005*"robert" + 0.005*"thanks" + 0.005*"amazing" + 0.005*"probably"
2021-02-20 23:43:04,226 : INFO : topic #2 (0.333): 0.005*"personal" + 0.005*"eat" + 0.005*"pain" + 0.005*"percent" + 0.005*"recommend" + 0.004*"remember" + 0.004*"career" + 0.004*"hit" + 0.004*"actually" + 0.004*"dad"
2021-02-20 23:43:04,227 : INFO : topic diff=0.003409, rho=0.147442
2021-02-20 23:43:51,028 : INFO : -6.247 per-word bound, 75.9 perplexity estimate based on a held-out corpus of 600 documents with 1724774 words
2021-02-20 23:43:51,028 : INFO : PROGRESS: pass 45, at document #600/600
2021-02-20 23:44:08,456 : INFO : topic #0 (0.333): 0.004*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"hold" + 0.004*"body" + 0.004*"issue" + 0.004*"middle" + 0.004*"single" + 0.004*"speaking" + 0.004*"instead"
2021-02-20 23:44:08,459 : INFO : topic #1 (0.333): 0.006*"terms" + 

In [121]:
lda1.print_topics()

2021-02-20 23:47:30,350 : INFO : topic #0 (0.333): 0.004*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"hold" + 0.004*"issue" + 0.004*"body" + 0.004*"middle" + 0.004*"single" + 0.004*"instead" + 0.004*"speaking"
2021-02-20 23:47:30,353 : INFO : topic #1 (0.333): 0.006*"terms" + 0.006*"supporting" + 0.006*"age" + 0.006*"talk" + 0.006*"second" + 0.006*"robert" + 0.006*"self" + 0.005*"thanks" + 0.005*"probably" + 0.005*"amazing"
2021-02-20 23:47:30,354 : INFO : topic #2 (0.333): 0.005*"personal" + 0.005*"eat" + 0.005*"pain" + 0.005*"percent" + 0.005*"recommend" + 0.004*"remember" + 0.004*"career" + 0.004*"hit" + 0.004*"actually" + 0.004*"dad"


[(0,
  '0.004*"dad" + 0.004*"guests" + 0.004*"kinds" + 0.004*"hold" + 0.004*"issue" + 0.004*"body" + 0.004*"middle" + 0.004*"single" + 0.004*"instead" + 0.004*"speaking"'),
 (1,
  '0.006*"terms" + 0.006*"supporting" + 0.006*"age" + 0.006*"talk" + 0.006*"second" + 0.006*"robert" + 0.006*"self" + 0.005*"thanks" + 0.005*"probably" + 0.005*"amazing"'),
 (2,
  '0.005*"personal" + 0.005*"eat" + 0.005*"pain" + 0.005*"percent" + 0.005*"recommend" + 0.004*"remember" + 0.004*"career" + 0.004*"hit" + 0.004*"actually" + 0.004*"dad"')]

In [ ]:
#topic space

In [26]:
corpus2 = matutils.Sparse2Corpus(_tfidf1)

In [28]:
id2word2 = dict((v, k) for k, v in tfidf1.vocabulary_.items())

In [32]:
lda2 = models.LdaModel(corpus=corpus2, num_topics=3, id2word=id2word2, passes=5)

2021-02-18 20:05:25,985 : INFO : using symmetric alpha at 0.3333333333333333
2021-02-18 20:05:25,986 : INFO : using symmetric eta at 0.3333333333333333
2021-02-18 20:05:25,995 : INFO : using serial LDA version on this node
2021-02-18 20:05:26,011 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 44569 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-18 20:05:26,023 : INFO : PROGRESS: pass 0, at document #2000/44569
2021-02-18 20:05:26,332 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:26,341 : INFO : topic #0 (0.333): 0.002*"1921" + 0.001*"40th" + 0.001*"1st" + 0.001*"221" + 0.001*"174" + 0.001*"3om" + 0.001*"150th" + 0.001*"2000s" + 0.001*"2019" + 0.001*"15"
2021-02-18 20:05:26,342 : INFO : topic #1 (0.333): 0.001*"1921" + 0.001*"353" + 0.001*"3x" + 0.001*"1955" + 0.001*"141" + 0.0

2021-02-18 20:05:28,968 : INFO : topic #0 (0.333): 0.001*"1944" + 0.001*"01" + 0.001*"164" + 0.001*"30th" + 0.001*"34" + 0.001*"113" + 0.001*"1920s" + 0.001*"1937" + 0.001*"1740" + 0.001*"238"
2021-02-18 20:05:28,970 : INFO : topic #1 (0.333): 0.000*"141" + 0.000*"130s" + 0.000*"266" + 0.000*"1800contacts" + 0.000*"1991" + 0.000*"337" + 0.000*"2001" + 0.000*"23" + 0.000*"140" + 0.000*"231"
2021-02-18 20:05:28,972 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"199" + 0.000*"220" + 0.000*"135" + 0.000*"237" + 0.000*"267" + 0.000*"2001" + 0.000*"32" + 0.000*"06"
2021-02-18 20:05:28,973 : INFO : topic diff=0.009811, rho=0.316228
2021-02-18 20:05:28,994 : INFO : PROGRESS: pass 0, at document #22000/44569
2021-02-18 20:05:29,174 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:29,183 : INFO : topic #0 (0.333): 0.001*"01" + 0.001*"178" + 0.001*"1944" + 0.001*"1989" + 0.001*"29th" + 0.001*"238" + 0.001*"1967" + 0.001*"2009" + 0.001*"

2021-02-18 20:05:31,448 : INFO : PROGRESS: pass 0, at document #40000/44569
2021-02-18 20:05:31,649 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:31,658 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"2200" + 0.001*"337" + 0.001*"295" + 0.001*"1937" + 0.001*"2026" + 0.001*"2000s" + 0.001*"1991" + 0.001*"34" + 0.001*"1976"
2021-02-18 20:05:31,659 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"23" + 0.000*"1800contacts" + 0.000*"141" + 0.000*"337" + 0.000*"113" + 0.000*"1991" + 0.000*"135"
2021-02-18 20:05:31,661 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"199" + 0.000*"237" + 0.000*"2001" + 0.000*"135" + 0.000*"267" + 0.000*"06" + 0.000*"337" + 0.000*"220"
2021-02-18 20:05:31,663 : INFO : topic diff=0.011192, rho=0.223607
2021-02-18 20:05:31,689 : INFO : PROGRESS: pass 0, at document #42000/44569
2021-02-18 20:05:31,880 : INFO : merging changes from 2000 documents into a model of 44569 docu

2021-02-18 20:05:33,714 : INFO : topic diff=0.004340, rho=0.202925
2021-02-18 20:05:33,734 : INFO : PROGRESS: pass 1, at document #14000/44569
2021-02-18 20:05:33,913 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:33,920 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"25" + 0.001*"237" + 0.001*"30th" + 0.001*"29th" + 0.001*"1972" + 0.001*"2nd" + 0.001*"223" + 0.001*"1985" + 0.001*"337"
2021-02-18 20:05:33,921 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"2014" + 0.000*"23" + 0.000*"3600" + 0.000*"337" + 0.000*"1925"
2021-02-18 20:05:33,923 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"06" + 0.000*"199" + 0.000*"2014" + 0.000*"3600" + 0.000*"220"
2021-02-18 20:05:33,924 : INFO : topic diff=0.003698, rho=0.202925
2021-02-18 20:05:33,945 : INFO : PROGRESS: pass 1, at document #16000/44569
2021-02-18 20:05:34,130 : INFO : mergi

2021-02-18 20:05:36,265 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"113" + 0.000*"1988" + 0.000*"2001" + 0.000*"135" + 0.000*"237" + 0.000*"06" + 0.000*"315"
2021-02-18 20:05:36,268 : INFO : topic diff=0.004669, rho=0.202925
2021-02-18 20:05:36,289 : INFO : PROGRESS: pass 1, at document #34000/44569
2021-02-18 20:05:36,464 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:36,472 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"1974" + 0.001*"1967" + 0.001*"141" + 0.001*"135" + 0.001*"337" + 0.001*"29th" + 0.001*"30" + 0.001*"2200" + 0.001*"1944"
2021-02-18 20:05:36,473 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"135" + 0.000*"113" + 0.000*"00000" + 0.000*"23" + 0.000*"35" + 0.000*"1640"
2021-02-18 20:05:36,476 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"135" + 0.000*"00000" + 0.000*"113" + 0.000*"35" + 0.000*"23" + 

...

2021-02-18 20:05:44,939 : INFO : merging changes from 569 documents into a model of 44569 documents
2021-02-18 20:05:44,952 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"2009" + 0.001*"141" + 0.001*"2029" + 0.001*"113" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"214" + 0.001*"1922"
2021-02-18 20:05:44,954 : INFO : topic #1 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"2004" + 0.000*"4000"
2021-02-18 20:05:44,955 : INFO : topic #2 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"2004" + 0.000*"4000"
2021-02-18 20:05:44,956 : INFO : topic diff=0.023632, rho=0.198872
2021-02-18 20:05:44,973 : INFO : PROGRESS: pass 3, at document #2000/44569
2021-02-18 20:05:45,151 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:45,159 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"141" 

2021-02-18 20:05:47,571 : INFO : PROGRESS: pass 3, at document #20000/44569
2021-02-18 20:05:47,778 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:47,785 : INFO : topic #0 (0.333): 0.001*"1944" + 0.001*"113" + 0.001*"30th" + 0.001*"01" + 0.001*"164" + 0.001*"1937" + 0.001*"34" + 0.001*"2009" + 0.001*"238" + 0.001*"2019"
2021-02-18 20:05:47,786 : INFO : topic #1 (0.333): 0.000*"2001" + 0.000*"130s" + 0.000*"2014" + 0.000*"237" + 0.000*"266" + 0.000*"278" + 0.000*"2060" + 0.000*"14ers" + 0.000*"113" + 0.000*"2200"
2021-02-18 20:05:47,788 : INFO : topic #2 (0.333): 0.000*"2001" + 0.000*"130s" + 0.000*"2014" + 0.000*"237" + 0.000*"266" + 0.000*"278" + 0.000*"2060" + 0.000*"14ers" + 0.000*"113" + 0.000*"2200"
2021-02-18 20:05:47,789 : INFO : topic diff=0.004024, rho=0.195052
2021-02-18 20:05:47,810 : INFO : PROGRESS: pass 3, at document #22000/44569
2021-02-18 20:05:48,027 : INFO : merging changes from 2000 documents into a model of 44569 docum

2021-02-18 20:05:50,142 : INFO : topic diff=0.002650, rho=0.195052
2021-02-18 20:05:50,627 : INFO : -8.579 per-word bound, 382.5 perplexity estimate based on a held-out corpus of 2000 documents with 686 words
2021-02-18 20:05:50,628 : INFO : PROGRESS: pass 3, at document #40000/44569
2021-02-18 20:05:50,810 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:50,817 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"2200" + 0.001*"337" + 0.001*"1937" + 0.001*"295" + 0.001*"2000s" + 0.001*"1991" + 0.001*"2026" + 0.001*"34" + 0.001*"208"
2021-02-18 20:05:50,818 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"414" + 0.000*"23" + 0.000*"237" + 0.000*"2014" + 0.000*"2007" + 0.000*"135" + 0.000*"113"
2021-02-18 20:05:50,820 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"414" + 0.000*"23" + 0.000*"237" + 0.000*"2014" + 0.000*"2007" + 0.000*"135" + 0.000*"113"
2021-02-18 20:05:50,821 : INFO : top

2021-02-18 20:05:52,685 : INFO : topic #2 (0.333): 0.000*"237" + 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"06" + 0.000*"2014" + 0.000*"23" + 0.000*"2x" + 0.000*"220" + 0.000*"31"
2021-02-18 20:05:52,686 : INFO : topic diff=0.004175, rho=0.191444
2021-02-18 20:05:52,705 : INFO : PROGRESS: pass 4, at document #14000/44569
2021-02-18 20:05:52,876 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:52,883 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"25" + 0.001*"237" + 0.001*"30th" + 0.001*"29th" + 0.001*"1972" + 0.001*"2nd" + 0.001*"223" + 0.001*"1985" + 0.001*"337"
2021-02-18 20:05:52,884 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"266" + 0.000*"2014" + 0.000*"383" + 0.000*"3600" + 0.000*"06" + 0.000*"31"
2021-02-18 20:05:52,885 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"266" + 0.000*"2014" + 0.000*"383" + 0.000*"3600" + 0.000*"06" + 0.000*"

2021-02-18 20:05:55,257 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"1988" + 0.000*"113" + 0.000*"2001" + 0.000*"135" + 0.000*"315" + 0.000*"237" + 0.000*"06"
2021-02-18 20:05:55,259 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"1988" + 0.000*"113" + 0.000*"2001" + 0.000*"135" + 0.000*"315" + 0.000*"237" + 0.000*"06"
2021-02-18 20:05:55,260 : INFO : topic diff=0.004434, rho=0.191444
2021-02-18 20:05:55,281 : INFO : PROGRESS: pass 4, at document #34000/44569
2021-02-18 20:05:55,454 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:55,462 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"1974" + 0.001*"1967" + 0.001*"141" + 0.001*"337" + 0.001*"135" + 0.001*"29th" + 0.001*"30" + 0.001*"1944" + 0.001*"2200"
2021-02-18 20:05:55,463 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"237" + 0.000*"135" + 0.000*"00000" + 0.000*"113" + 0.000*"35" + 0.000*"23" + 

In [34]:
lda2.print_topics()

2021-02-18 20:05:57,444 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"2009" + 0.001*"113" + 0.001*"141" + 0.001*"2029" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"1944" + 0.001*"1922"
2021-02-18 20:05:57,447 : INFO : topic #1 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"
2021-02-18 20:05:57,449 : INFO : topic #2 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"


[(0,
  '0.001*"2024" + 0.001*"2009" + 0.001*"113" + 0.001*"141" + 0.001*"2029" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"1944" + 0.001*"1922"'),
 (1,
  '0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"'),
 (2,
  '0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"')]

In [ ]:
#topic space